In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ADMIN\\Documents\\Data Analytics\\Machine Learning\\ML Projects\\End_End_Project_With_MLFlow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ADMIN\\Documents\\Data Analytics\\Machine Learning\\ML Projects\\End_End_Project_With_MLFlow'

In [5]:
#Defining the Data Class Entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

dataclass from the dataclasses module: Used to define a class that holds data with minimal boilerplate.
Path from the pathlib module: Provides an object-oriented way to handle filesystem paths.

Has four fields:
- root_dir: The root directory where data will be managed.
- source_URL: The URL from which data will be downloaded.
- local_data_file: The local file path where the downloaded data will be stored.
- unzip_dir: The directory where the downloaded file will be unzipped.

In [6]:
from mlProject.constants import * #import everything
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        #Fetches the data_ingestion section from self.config.
        config = self.config.data_ingestion

        #Creates the root directory for data ingestion.
        create_directories([config.root_dir])

        #Instantiates a DataIngestionConfig object using the fetched configuration values.
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )
        #Returns the DataIngestionConfig object.
        return data_ingestion_config

- The __init__ method takes three file paths as optional parameters for configuration, parameters, and schema files.
- Reads these YAML files using a hypothetical read_yaml function and assigns them to self.config, self.params, and self.schema.
- Creates directories as specified by self.config.artifacts_root.

In [8]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        #Checks if the local data file already exists.
        #If not, downloads the file from self.config.source_URL and saves it to self.config.local_data_file
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        #Extracts the contents of the zip file located at self.config.local_data_file to the unzip_path.
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

- The __init__ method takes a DataIngestionConfig object as a parameter and assigns it to self.config.


In [11]:
try:
    #Creates an instance of ConfigurationManager which reads the configuration files.
    config = ConfigurationManager()
    #Calls get_data_ingestion_config to get a DataIngestionConfig object.
    data_ingestion_config = config.get_data_ingestion_config()
    #Creates an instance of DataIngestion with the configuration.
    data_ingestion = DataIngestion(config=data_ingestion_config)
    #Calls download_file to download the data file.
    data_ingestion.download_file()
    #Calls extract_zip_file to extract the contents of the downloaded zip file.
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-17 12:26:04,014: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-17 12:26:04,018: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-17 12:26:04,021: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-17 12:26:04,023: INFO: common: created directory at: artifacts]
[2024-07-17 12:26:04,025: INFO: common: created directory at: artifacts/data_ingestion]
[2024-07-17 12:26:05,510: INFO: 2366278714: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 92FC:18B97A:2194DD:351839:66978E2D
Accept-Ranges: bytes
Date: Wed, 17